In [ ]:
import numpy as np, cv2, math
from PIL import Image
from pathlib import Path
from rembg import remove

IMG_EXTS = {".jpg",".jpeg",".png",".bmp",".webp",".JPG",".JPEG",".PNG"}

def largest_component_bbox(mask: np.ndarray, min_area_ratio=0.002):
    # mask: 0/255
    num, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    if num <= 1:
        return None
    H, W = mask.shape
    area_min = H*W*min_area_ratio
    lid, best = None, 0
    for i in range(1, num):
        a = stats[i, cv2.CC_STAT_AREA]
        if a >= area_min and a > best:
            lid, best = i, a
    if lid is None:
        return None
    x, y, w, h, _ = stats[lid]
    return x, y, x+w, y+h

def expand_box(x1,y1,x2,y2, pad_ratio, W,H):
    w, h = x2-x1, y2-y1
    px, py = int(w*pad_ratio), int(h*pad_ratio)
    return max(0,x1-px), max(0,y1-py), min(W,x2+px), min(H,y2+py)

def letterbox_resize_rgb(rgb: np.ndarray, size=512, fill=(255,255,255)):
    H, W = rgb.shape[:2]
    s = min(size/W, size/H)
    nw, nh = int(W*s), int(H*s)
    resized = cv2.resize(rgb, (nw, nh), interpolation=cv2.INTER_CUBIC)
    canvas = np.full((size, size, 3), fill, dtype=np.uint8)
    x0, y0 = (size-nw)//2, (size-nh)//2
    canvas[y0:y0+nh, x0:x0+nw] = resized
    return canvas

def auto_crop_rembg(img_path: Path, pad_ratio=0.08, out_size=512, save_debug=None):
    im = Image.open(img_path).convert("RGBA")
    rgba = remove(im)

    # 1. 알파 채널
    alpha = np.array(rgba.split()[-1])

    # 2. 테두리 다듬기
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    alpha = cv2.erode(alpha, kernel, iterations=1)
    alpha = cv2.GaussianBlur(alpha,(5,5),0)

    # 3. 알파 적용한 RGB
    rgb = np.array(rgba)[:,:,:3]
    mask = alpha.astype(np.float32)/255.0
    rgb = (rgb*mask[...,None] + 255*(1-mask[...,None])).astype(np.uint8)

    # 4. 마스크로 bbox 추출
    mask_bin = (alpha > 0).astype(np.uint8)*255
    bbox = largest_component_bbox(mask_bin, min_area_ratio=0.001)
    if bbox is None:
        return None

    x1,y1,x2,y2 = expand_box(*bbox, pad_ratio, rgb.shape[1], rgb.shape[0])
    crop = rgb[y1:y2, x1:x2]
    return letterbox_resize_rgb(crop, size=out_size)


In [ ]:
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Malgun Gothic'  # 맑은 고딕
plt.rcParams['axes.unicode_minus'] = False     # 마이너스 기호 깨짐 방지

input_dir = Path("data/Dataset_project4/플라스틱PE")
test_img = input_dir / "152060@1_04001_220811_P1_T1.jpg"   # 원하는 파일 이름


# 원본
row_img = Image.open(test_img).convert("RGB")

# 크롭 결과
res = auto_crop_rembg(test_img, pad_ratio=0.08, out_size=512, save_debug="data/debug_masks")

print("테스트 파일:", test_img)

if res is None:
    print("rembg에서도 전경 감지 실패 → debug_masks에 마스크 저장했음.")
else:
    # 1행 2열 subplot에 원본 + 결과를 나란히 출력
    plt.figure(figsize=(10,5))

    plt.subplot(1,2,1)
    plt.imshow(row_img)
    plt.title("원본 이미지")
    plt.axis("off")

    plt.subplot(1,2,2)
    plt.imshow(res)
    plt.title("크롭 결과")
    plt.axis("off")

    plt.show()


In [ ]:
from pathlib import Path
from PIL import Image
from tqdm import tqdm

input_root  = Path("data/Dataset_project4")        # 원본 루트
output_root = Path("data/Dataset_project4_crop")   # 결과 루트
output_root.mkdir(parents=True, exist_ok=True)

# 제외할 클래스(폴더)들
exclude_classes = {"비닐"}

# 어떤 클래스가 처리 대상인지 확인
all_classes = sorted([p.name for p in input_root.iterdir() if p.is_dir()])
target_classes = [c for c in all_classes if c not in exclude_classes]
print("처리 대상 클래스:", target_classes)
print("제외 클래스:", list(exclude_classes))

ok, fail = 0, 0

# 모든 하위 파일 순회
files = [p for p in input_root.rglob("*") if p.suffix.lower() in IMG_EXTS]
for img_path in tqdm(files):
    rel = img_path.relative_to(input_root)
    # 최상위 폴더명이 클래스명이라고 가정
    class_name = rel.parts[0] if len(rel.parts) > 0 else ""
    if class_name in exclude_classes:
        continue

    out_path = output_root / rel
    out_path.parent.mkdir(parents=True, exist_ok=True)

    out = auto_crop_rembg(img_path, pad_ratio=0.08, out_size=512)
    if out is None:
        fail += 1
        # 필요하면 디버그용 마스크 저장 옵션을 auto_crop_rembg에 켜서 원인 파악
        # print("fail:", img_path)
        continue

    Image.fromarray(out).save(out_path.with_suffix(".jpg"), quality=95)
    ok += 1

print(f"완료: success={ok}, fail={fail}, 저장 위치={output_root}")
